# Satcking Logistic Regression with Catboost

In [ ]:
train_processed, test_processed = preprocess_and_detect_uid(train_df, test_df, train_identity, test_identity)
test_processed.columns = test_processed.columns.str.replace('-', '_')
train_processed.replace([np.inf, -np.inf], np.nan, inplace=True)
test_processed.replace([np.inf, -np.inf], np.nan, inplace=True)

Starting preprocessing...
Mem. usage decreased to 542.35 Mb (0.0% reduction)
Mem. usage decreased to 472.59 Mb (0.0% reduction)
Preprocessing and UID detection complete.


In [ ]:
filled_train, filled_test, train_y, cat_features = preprocess_for_catboost(train_processed, test_processed)

print("Preprocessed Train Shape:", filled_train.shape)
print("Preprocessed Test Shape:", filled_test.shape)
print("Categorical Features:", cat_features)


Preprocessed Train Shape: (590540, 65)
Preprocessed Test Shape: (506691, 65)
Categorical Features: [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]


In [ ]:
filled_train['stacked_log_reg'] = predicted_proba_train_logistic_Regression_baseline_for_Stacking
filled_test['stacked_log_reg'] = predicted_proba_test_logistic_Regression_baseline_for_Stacking

In [ ]:
filled_train['stacked_log_reg']

,stacked_log_reg
0,0.751780
1,0.993111
2,0.022570
3,0.209198
4,0.998900
...,...
590535,0.000249
590536,0.019864
590537,0.003043
590538,0.009289


In [ ]:
import optuna
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Target variable
y_train = train_processed['isFraud']

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters for CatBoost
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.5, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 10.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "loss_function": "Logloss",
        "eval_metric": "Logloss",
        "task_type": "GPU",
        "verbose": False,
        "random_state": 42,
    }

    # Cross-validation setup
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    auc_scores = []

    for train_idx, val_idx in cv.split(filled_train, y_train):
        X_train, X_val = filled_train.iloc[train_idx], filled_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Create CatBoost Pool
        train_pool = Pool(X_train, y_train_fold, cat_features=cat_features)
        val_pool = Pool(X_val, y_val_fold, cat_features=cat_features)

        # Train CatBoost model
        model = CatBoostClassifier(**params)
        model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=False)

        # Predict probabilities for validation set
        y_val_pred_proba = model.predict_proba(X_val)[:, 1]  # Probabilities for class 1

        # Calculate AUC
        auc = roc_auc_score(y_val_fold, y_val_pred_proba)
        auc_scores.append(auc)
        print(np.mean(auc_scores))

    # Return the mean AUC (maximize this)
    return np.mean(auc_scores)

# Run Optuna optimization
study = optuna.create_study(direction="maximize", study_name="CatBoost AUC Tuning")
study.optimize(objective, n_trials=50)

# Print best parameters
print("Best parameters:", study.best_trial.params)



[I 2024-12-08 16:48:59,080] A new study created in memory with name: CatBoost AUC Tuning


0.9589560358013585
0.9609716040234281
0.960934019884608
0.9603944105305997


[I 2024-12-08 16:51:50,362] Trial 0 finished with value: 0.9607153943222896 and parameters: {'iterations': 557, 'depth': 6, 'learning_rate': 0.45326398619337, 'l2_leaf_reg': 0.011476656803977863, 'bagging_temperature': 0.2240223010411081, 'random_strength': 5.536505896148168, 'border_count': 169}. Best is trial 0 with value: 0.9607153943222896.


0.9607153943222896
0.9351211470888625
0.9362380964563046
0.936397239262857
0.9368187776196131


[I 2024-12-08 16:53:00,744] Trial 1 finished with value: 0.9368149481176452 and parameters: {'iterations': 246, 'depth': 3, 'learning_rate': 0.10021798811709581, 'l2_leaf_reg': 0.03275453222778487, 'bagging_temperature': 0.45006997858307973, 'random_strength': 0.23060323305153618, 'border_count': 78}. Best is trial 0 with value: 0.9607153943222896.


...

0.8621720592401136
0.9617858103107048
0.9635313895416986
0.9637594880346508
0.9642038886434068


[I 2024-12-08 20:33:57,723] Trial 48 finished with value: 0.9643102869751174 and parameters: {'iterations': 797, 'depth': 8, 'learning_rate': 0.18200937937524891, 'l2_leaf_reg': 0.28310069591566106, 'bagging_temperature': 0.9797257026522773, 'random_strength': 2.8241524957938022, 'border_count': 99}. Best is trial 43 with value: 0.9680341026849497.


0.9643102869751174
0.9459060108678643
0.9472824942689315
0.9476508078333951
0.9483578692198142


[I 2024-12-08 20:36:43,067] Trial 49 finished with value: 0.9487187224402718 and parameters: {'iterations': 700, 'depth': 4, 'learning_rate': 0.04690662989318129, 'l2_leaf_reg': 0.13180821967242326, 'bagging_temperature': 0.06729213916605695, 'random_strength': 2.2040551445135046, 'border_count': 129}. Best is trial 43 with value: 0.9680341026849497.


0.9487187224402718
Best parameters: {'iterations': 958, 'depth': 7, 'learning_rate': 0.13855689881749275, 'l2_leaf_reg': 0.6113539009883876, 'bagging_temperature': 0.17710574014981362, 'random_strength': 1.3229705738948407, 'border_count': 62}


In [ ]:
# Train the final model with the best parameters on the full dataset
#best_params = study.best_trial.params
best_params = {'iterations': 958, 'depth': 7, 'learning_rate': 0.13855689881749275, 'task_type': 'GPU', 'l2_leaf_reg': 0.6113539009883876, 'bagging_temperature': 0.17710574014981362, 'random_strength': 1.3229705738948407, 'border_count': 62}
final_pool = Pool(filled_train, y_train, cat_features=cat_features)
final_model = CatBoostClassifier(**best_params, verbose=100)
final_model.fit(final_pool)

# Make predictions on the test set
test_pool = Pool(filled_test, cat_features=cat_features)
test_pred_proba = final_model.predict_proba(test_pool)[:, 1]

# Create submission
submission = pd.DataFrame({
    "TransactionID": test_processed["TransactionID"],
    "isFraud": test_pred_proba
})
submission.to_csv("catboost_log_reg_stacked_auc_submission.csv", index=False)

print("Submission file created: catboost_auc_submission.csv")

0:	learn: 0.4893707	total: 79.7ms	remaining: 1m 16s
100:	learn: 0.0710812	total: 7.17s	remaining: 1m
200:	learn: 0.0635219	total: 14.8s	remaining: 55.6s
300:	learn: 0.0593398	total: 22.3s	remaining: 48.6s
400:	learn: 0.0564467	total: 29.4s	remaining: 40.9s
500:	learn: 0.0546512	total: 36.7s	remaining: 33.5s
600:	learn: 0.0525618	total: 44s	remaining: 26.1s
700:	learn: 0.0507231	total: 51.1s	remaining: 18.7s
800:	learn: 0.0490758	total: 58.3s	remaining: 11.4s
900:	learn: 0.0476297	total: 1m 5s	remaining: 4.15s
957:	learn: 0.0468720	total: 1m 9s	remaining: 0us
Submission file created: catboost_auc_submission.csv


In [ ]:
filled_test['stacked_log_reg']

,stacked_log_reg
0,0.999997
1,0.999885
2,0.999993
3,1.000000
4,1.000000
...,...
506686,1.000000
506687,0.999928
506688,0.994934
506689,0.999972


In [ ]:
import optuna
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

# Target variable
y_train = train_processed['isFraud']

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters for CatBoost
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.5, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 10.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "loss_function": "Logloss",
        "eval_metric": "Logloss",
        "task_type": "GPU",
        "verbose": False,
        "random_state": 42,
    }

    # Cross-validation setup
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    roc_auc_scores = []
    pr_auc_scores = []

    for train_idx, val_idx in cv.split(filled_train, y_train):
        X_train, X_val = filled_train.iloc[train_idx], filled_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Create CatBoost Pool
        train_pool = Pool(X_train, y_train_fold, cat_features=cat_features)
        val_pool = Pool(X_val, y_val_fold, cat_features=cat_features)

        # Train CatBoost model
        model = CatBoostClassifier(**params)
        model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=False)

        # Predict probabilities for validation set
        y_val_pred_proba = model.predict_proba(X_val)[:, 1]  # Probabilities for class 1

        # Calculate ROC AUC
        roc_auc = roc_auc_score(y_val_fold, y_val_pred_proba)
        roc_auc_scores.append(roc_auc)

        # Calculate Precision-Recall AUC
        precision, recall, _ = precision_recall_curve(y_val_fold, y_val_pred_proba)
        pr_auc = auc(recall, precision)
        pr_auc_scores.append(pr_auc)

    # Save metrics for this trial
    trial.set_user_attr("roc_auc_scores", roc_auc_scores)
    trial.set_user_attr("pr_auc_scores", pr_auc_scores)
    print(f'ROC AUC: {np.mean(roc_auc_scores)}')
    print(f'PR AUC: {np.mean(pr_auc_scores)}')
    # Return the mean ROC AUC and PR AUC
    return np.mean(roc_auc_scores), np.mean(pr_auc_scores)

# Run Optuna multi-objective optimization
study = optuna.create_study(
    directions=["maximize", "maximize"],  # Multi-objective optimization
    study_name="CatBoost Multi-Objective Tuning"
)
study.optimize(objective, n_trials=50)

# Save metrics to DataFrame for later analysis
results = []
for trial in study.trials:
    results.append({
        "trial_number": trial.number,
        "roc_auc": np.mean(trial.user_attrs["roc_auc_scores"]),
        "pr_auc": np.mean(trial.user_attrs["pr_auc_scores"])
    })

metrics_df = pd.DataFrame(results)
metrics_df.to_csv("optuna_metrics.csv", index=False)

# Print best parameters
print("Best ROC AUC Trial:")
print(study.best_trials[0].values)
print("Best PR AUC Trial:")
print(study.best_trials[1].values)


[I 2024-12-08 21:43:24,296] A new study created in memory with name: CatBoost Multi-Objective Tuning
[I 2024-12-08 21:50:20,342] Trial 0 finished with values: [0.9644038163549361, 0.8325089024842246] and parameters: {'iterations': 630, 'depth': 10, 'learning_rate': 0.1535726401767315, 'l2_leaf_reg': 0.24527383446914192, 'bagging_temperature': 0.7758189094916633, 'random_strength': 1.9454538767507445, 'border_count': 52}.


ROC AUC: 0.9644038163549361
PR AUC: 0.8325089024842246


[I 2024-12-08 21:52:22,880] Trial 1 finished with values: [0.8938027582853069, 0.5408833680578891] and parameters: {'iterations': 679, 'depth': 3, 'learning_rate': 0.004845294259107338, 'l2_leaf_reg': 0.3721982120361202, 'bagging_temperature': 0.8158767212007675, 'random_strength': 0.45685457709192967, 'border_count': 32}.


ROC AUC: 0.8938027582853069
PR AUC: 0.5408833680578891


[I 2024-12-08 21:58:30,692] Trial 2 finished with values: [0.9407119671752021, 0.7129449967594199] and parameters: {'iterations': 966, 'depth': 8, 'learning_rate': 0.013284223756845825, 'l2_leaf_reg': 1.1370348208471195, 'bagging_temperature': 0.9228547607338592, 'random_strength': 8.826745757097349, 'border_count': 107}.


...

ROC AUC: 0.955326458425492
PR AUC: 0.7760591703501679


[I 2024-12-09 00:54:02,167] Trial 48 finished with values: [0.8605575379521021, 0.39863332067479806] and parameters: {'iterations': 176, 'depth': 3, 'learning_rate': 0.01326700333907043, 'l2_leaf_reg': 2.256086049670873, 'bagging_temperature': 0.8732120292014017, 'random_strength': 4.885625452841248, 'border_count': 53}.


ROC AUC: 0.8605575379521021
PR AUC: 0.39863332067479806


[I 2024-12-09 00:54:46,077] Trial 49 finished with values: [0.7709100028733026, 0.18291348708084565] and parameters: {'iterations': 147, 'depth': 4, 'learning_rate': 0.00358964153892542, 'l2_leaf_reg': 0.006846231219828737, 'bagging_temperature': 0.15211588131822706, 'random_strength': 7.5419443447312045, 'border_count': 175}.


ROC AUC: 0.7709100028733026
PR AUC: 0.18291348708084565
Best ROC AUC Trial:
[0.9667078513937983, 0.8365462137252907]
Best PR AUC Trial:
[0.9669135604802722, 0.8305425966913023]
